In [17]:
# 📦 Import Libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error

In [19]:

train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

# 🎯 Separate features and target
X = train.drop(columns=["id", "Calories"])
y = train["Calories"]
X_test = test.drop(columns=["id"])

# 🔄 One-Hot Encoding for categorical variables
X_encoded = pd.get_dummies(X, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# ⚙️ Align test set to training set columns
X_test_encoded = X_test_encoded.reindex(columns=X_encoded.columns, fill_value=0)


In [20]:
# 🔀 Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# 🌲 Random Forest with GridSearchCV
rf = RandomForestRegressor(random_state=42)



In [21]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_log_error',
    verbose=1,
    n_jobs=-1
)

In [ ]:

# 🚂 Train the model
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_


Fitting 3 folds for each of 24 candidates, totalling 72 fits


In [ ]:

# ✅ Predict and Evaluate on Validation Set
y_val_pred = best_model.predict(X_val)

In [ ]:

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))


print("✅ Best Parameters:", grid_search.best_params_)
print("📉 Validation RMSLE:", rmsle(y_val, y_val_pred))

In [ ]:
# 🧪 Predict on Test Set
y_test_pred = best_model.predict(X_test_encoded)

# 📤 Prepare Submission File
submission = pd.DataFrame({
    "id": test["id"],
    "Calories": y_test_pred
})
submission.to_csv("submission.csv", index=False)

print("🚀 Submission file created: submission.csv")